
You might want to consider the [start](search.ipynb) of this tutorial.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use

In [3]:
VERSION = '2017'

In [4]:
A = use('bhsa', hoist=globals(), version=VERSION)

Using bhsa commit 8738a528b7388189f8654a3bdf47f8d070ac3f6c
  in C:\Users\oglan/text-fabric-data/__apps__/bhsa
Using etcbc/bhsa/tf - 2017 rv1.6 in C:\Users\oglan/text-fabric-data
Using etcbc/phono/tf - 2017 r1.2 in C:\Users\oglan/text-fabric-data
Using etcbc/parallels/tf - 2017 r1.2 in C:\Users\oglan/text-fabric-data


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://github.com/annotation/app-bhsa" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 7.3.0</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Use/Search/" title="Search Templates Introduction and Reference">Search Reference</a>

## Rough edges

It might be helpful to peek under the hood, especially when exploring searches that go slow.

If you went through the previous parts of the tutorial you have encountered cases where things come
to a grinding halt.

Yet we can get a hunch of what is going on, even in those cases.
For that, we use the lower-level search api `S` of Text-Fabric, and not the 
wrappers that the corpus specific `A` api provides.

The main difference is, that `S.search()` returns a *generator* of the results, 
whereas `A.search()` returns a list of the results.
In fact, `A.search()` calls the generator function delivered by `S.search()` as often as needed.

For some queries, the fetching of results is quite costly, so costly that we do not want to fetch
all results up-front. Rather we want to fetch a few, to see how it goes.
In these cases, directly using `S.search()` is preferred over `A.search()`.

In [5]:
query = '''
book
  chapter
    verse
      phrase det=und
        word lex=>LHJM/
'''

### Study

First we call `S.study(query)`.

The syntax will be checked, features loaded, the search space will be set up, narrowed down, 
and the fetching of results will be prepared, but not yet executed.

In order to make the query a bit more interesting, we lift the constraint that the results must be in Genesis 1-2.

In [6]:
S.study(query)

   |     0.00s Feature overview: 111 for nodes; 8 for edges; 2 configs; 7 computed
  0.00s Checking search template ...
  0.00s Setting up search space for 5 objects ...
  0.54s Constraining search space with 4 relations ...
  0.55s Setting up retrieval plan ...
  0.57s Ready to deliver results from 2735 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


Before we rush to the results, lets have a look at the *plan*.

In [7]:
S.showPlan()

  8.15s The results are connected to the original search template as follows:
 0     
 1 R0  book
 2 R1    chapter
 3 R2      verse
 4 R3        phrase det=und
 5 R4          word lex=>LHJM/
 6     


Here you see already what your results will look like.
Each result `r` is a *tuple* of nodes:
```
(R0, R1, R2, R3, R4)
```
that instantiate the objects in your template.

In case you are curious, you can get details about the search space as well:

In [8]:
S.showPlan(details=True)

Search with 5 objects and 4 relations
Results are instantiations of the following objects:
node  0-book                              (    29   choices)
node  1-chapter                           (   329   choices)
node  2-verse                             (   754   choices)
node  3-phrase                            (   805   choices)
node  4-word                              (   818   choices)
Instantiations are computed along the following relations:
node                      0-book          (    29   choices)
edge  0-book          [[  1-chapter       (     9.0 choices)
edge  1-chapter       [[  2-verse         (     1.7 choices)
edge  2-verse         [[  3-phrase        (     1.1 choices)
edge  3-phrase        [[  4-word          (     1.1 choices)
    13s The results are connected to the original search template as follows:
 0     
 1 R0  book
 2 R1    chapter
 3 R2      verse
 4 R3        phrase det=und
 5 R4          word lex=>LHJM/
 6     


The part about the *nodes* shows you how many possible instantiations for each object in your template
has been found.
These are not results yet, because only combinations of instantiations
that satisfy all constraints are results.

The constraints come from the relations between the objects that you specified.
In this case, there is only an implicit relation: embedding `[[`. 
Later on we'll examine all basic relations.

The part about the *edges* shows you the constraints,
and in what order they will be computed when stitching results together.
In this case the order is exactly the order by which the relations appear in the template,
but that will not always be the case.
Text-Fabric spends some time and ingenuity to find out an optimal *stitch plan*.
Fetching results is like selecting a node, stitching it to another node with an edge,
and so on, until a full stitch of nodes intersects with all the node sets from which they
must be chosen (the yarns).

Fetching results may take time. 

For some queries, it can take a large amount of time to walk through all results.
Even worse, it may happen that it takes a large amount of time before getting the *first* result.
During stitching, many stitchings will be tried and fail before they can be completed.

This has to do with search strategies on the one hand,
and the very likely possibility to encounter *pathological* search patterns,
which have billions of results, mostly unintended.
For example, a simple query that asks for 5 words in the Hebrew Bible without further constraints,
will have 425,000 to the power of 5 results.
That is 10-e28 (a one with 28 zeros),
roughly the number of molecules in a few hundred liters of air.
That may not sound much, but it is 10,000 times the amount of bytes
that can be currently stored on the whole Internet.

Text-Fabric search is not yet done with finding optimal search strategies,
and I hope to refine its arsenal of methods in the future, depending on what you report.

### Counting results
It is always a good idea to get a feel for the amount of results, before you dive into them head-on.

In [9]:
S.count(progress=1, limit=5)

  0.00s Counting results per 1 up to 5 ...
   |     0.00s 1
   |     0.00s 2
   |     0.00s 3
   |     0.00s 4
   |     0.01s 5
  0.01s Done: 5 results


We asked for 5 results in total, with a progress message for every one.
That was a bit conservative.

In [10]:
S.count(progress=100, limit=500)

  0.00s Counting results per 100 up to 500 ...
   |     0.01s 100
   |     0.02s 200
   |     0.03s 300
   |     0.05s 400
   |     0.06s 500
  0.06s Done: 500 results


Still pretty quick, now we want to count all results.

In [11]:
S.count(progress=200, limit=-1)

  0.00s Counting results per 200 up to  the end of the results ...
   |     0.02s 200
   |     0.04s 400
   |     0.06s 600
   |     0.07s 800
  0.08s Done: 818 results


### Fetching results

It is time to see something of those results.

In [12]:
S.fetch(limit=10)

((426585, 426624, 1414190, 651505, 4),
 (426585, 426624, 1414191, 651515, 26),
 (426585, 426624, 1414192, 651520, 34),
 (426585, 426624, 1414193, 651528, 42),
 (426585, 426624, 1414193, 651534, 50),
 (426585, 426624, 1414194, 651538, 60),
 (426585, 426624, 1414195, 651554, 81),
 (426585, 426624, 1414196, 651564, 97),
 (426585, 426624, 1414197, 651578, 127),
 (426585, 426624, 1414198, 651590, 142))

Not very informative.
Just a quick observation: look at the last column.
These are the result nodes for the `word` part in the query, indicated as `R7` by `showPlan()` before.
And indeed, they are all below 425,000, the number of words in the Hebrew Bible.

Nevertheless, we want to glean a bit more information off them.

In [13]:
for r in S.fetch(limit=10):
    print(S.glean(r))

  Genesis 1:1 phrase[אֱלֹהִ֑ים ] אֱלֹהִ֑ים 
  Genesis 1:2 phrase[ר֣וּחַ אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:3 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:4 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:4 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:5 phrase[אֱלֹהִ֤ים׀ ] אֱלֹהִ֤ים׀ 
  Genesis 1:6 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:7 phrase[אֱלֹהִים֮ ] אֱלֹהִים֮ 
  Genesis 1:8 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:9 phrase[אֱלֹהִ֗ים ] אֱלֹהִ֗ים 


##### Caution
> It is not possible to do `len(S.fetch())`.
Because `fetch()` is a *generator*, not a list.
It will deliver a result every time it is being asked and for as long as there are results,
but it does not know in advance how many there will be.

>Fetching a result can be costly, because due to the constraints, a lot of possibilities
may have to be tried and rejected before a the next result is found.

> That is why you often see results coming in at varying speeds when counting them.

We can also use `A.table()` to make a list of results.
This function is part of the `Bhsa` API, not of the generic Text-Fabric machinery, as opposed to `S.glean()`.

So, you can use `S.glean()` for every Text-Fabric corpus, but the output is still not very nice.
`A.table()` gives much nicer output, but works only for the BHSA corpus.

We put hyperlinks to SHEBANQ under column 3.

In [14]:
A.table(S.fetch(limit=10), linked=3)

n | p | book | chapter | verse | phrase | word
--- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1">Genesis 1:1</a>בְּרֵאשִׁ֖ית בָּרָ֣א <span  class="hl hlup" >אֱלֹהִ֑ים </span>אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a>וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְ<span  class="hlup" >ר֣וּחַ </span><span  class="hl hlup" >אֱלֹהִ֔ים </span>מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </span>|<span class="hb"><span  class="hlup" >ר֣וּחַ </span><span  class="hl hlup" >אֱלֹהִ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֔ים </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:3">Genesis 1:3</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3">Genesis 1:3</a>וַיֹּ֥אמֶר <span  class="hl hlup" >אֱלֹהִ֖ים </span>יְהִ֣י אֹ֑ור וַֽיְהִי־אֹֽור׃ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֖ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֖ים </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:4">Genesis 1:4</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא <span  class="hl hlup" >אֱלֹהִ֛ים </span>אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֛ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֛ים </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:4">Genesis 1:4</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל <span  class="hl hlup" >אֱלֹהִ֔ים </span>בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֔ים </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:5">Genesis 1:5</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:5" sec="Genesis 1:5">Genesis 1:5</a>וַיִּקְרָ֨א <span  class="hl hlup" >אֱלֹהִ֤ים׀ </span>לָאֹור֙ יֹ֔ום וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום אֶחָֽד׃ פ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֤ים׀ </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֤ים׀ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:6">Genesis 1:6</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:6" sec="Genesis 1:6">Genesis 1:6</a>וַיֹּ֣אמֶר <span  class="hl hlup" >אֱלֹהִ֔ים </span>יְהִ֥י רָקִ֖יעַ בְּתֹ֣וךְ הַמָּ֑יִם וִיהִ֣י מַבְדִּ֔יל בֵּ֥ין מַ֖יִם לָמָֽיִם׃ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֔ים </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:7">Genesis 1:7</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">Genesis 1:7</a>וַיַּ֣עַשׂ <span  class="hl hlup" >אֱלֹהִים֮ </span>אֶת־הָרָקִיעַ֒ וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ וּבֵ֣ין הַמַּ֔יִם אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִים֮ </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִים֮ </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:8">Genesis 1:8</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:8" sec="Genesis 1:8">Genesis 1:8</a>וַיִּקְרָ֧א <span  class="hl hlup" >אֱלֹהִ֛ים </span>לָֽרָקִ֖יעַ שָׁמָ֑יִם וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום שֵׁנִֽי׃ פ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֛ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֛ים </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:9">Genesis 1:9</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:9" sec="Genesis 1:9">Genesis 1:9</a>וַיֹּ֣אמֶר <span  class="hl hlup" >אֱלֹהִ֗ים </span>יִקָּו֨וּ הַמַּ֜יִם מִתַּ֤חַת הַשָּׁמַ֨יִם֙ אֶל־מָקֹ֣ום אֶחָ֔ד וְתֵרָאֶ֖ה הַיַּבָּשָׁ֑ה וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֗ים </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֗ים </span></span>

## Slow queries

The search template above has some pretty tight constraints on one of its objects,
so the amount of data to deal with is pretty limited.

If the constraints are weak, search may become slow.

For example:

In [15]:
query = '''
% test
% verse book=Genesis chapter=2 verse=25
verse
  clause
                                 
    p1:phrase
        w1:word
        w3:word
        w1 < w3

    p2:phrase
        w2:word
        w1 < w2 
        w3 > w2
    
    p1 < p2   
'''

A couple of remarks you may have encountered before.

* some objects have got a name
* there are additional relations specified between named objects
* `<` means: *comes before*, and `>`: *comes after* in the canonical order for nodes,
  which for words means: comes textually before/after, but for other nodes the meaning
  is explained [here](https://dans-labs.github.io/text-fabric/Api/General/#navigating-nodes)
* later on we describe those relations in more detail

> **Note on order**
Look at the words `w1` and `w3` below phrase `p1`.
Although in the template `w1` comes before `w3`, this is not 
translated in a search constraint of the same nature.

> Order between objects in a template is never significant, only embedding is.

Because order is not significant, you have to specify order yourself, using relations.

It turns out that this is better than the other way around.
In MQL order *is* significant, and it is very difficult to 
search for `w1` and `w2` in any order.
Especially if your are looking for more than 2 complex objects with lots of feature
conditions, your search template would explode if you had to spell out all
possible permutations. See the example of Reinoud Oosting below.

> **Note on gaps**
Look at the phrases `p1` and `p2`.
We do not specify an order here, only that they are different.
In order to prevent duplicated searches with `p1` and `p2` interchanged, we even 
stipulate that `p1 < p2`.
There are many spatial relationships possible between different objects.
In many cases, neither the one comes before the other, nor vice versa.
They can overlap, one can occur in a gap of the other, they can be completely disjoint
and interleaved, etc.

In [16]:
S.study(query)

   |     0.00s Feature overview: 111 for nodes; 8 for edges; 2 configs; 7 computed
  0.00s Checking search template ...
  0.00s Setting up search space for 7 objects ...
  0.26s Constraining search space with 10 relations ...
  0.30s Setting up retrieval plan ...
  0.34s Ready to deliver results from 1897440 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


That was quick!
Well, Text-Fabric knows that narrowing down the search space in this case would take ages,
without resulting in a significantly shrunken space.
So it skips doing so for most constraints.

Let us see the plan, with details.

In [17]:
S.showPlan(details=True)

Search with 7 objects and 10 relations
Results are instantiations of the following objects:
node  0-verse                             ( 23213   choices)
node  1-clause                            ( 88101   choices)
node  2-phrase                            (253187   choices)
node  3-word                              (426584   choices)
node  4-word                              (426584   choices)
node  5-phrase                            (253187   choices)
node  6-word                              (426584   choices)
Instantiations are computed along the following relations:
node                      0-verse         ( 23213   choices)
edge  0-verse         [[  1-clause        (     4.1 choices)
edge  1-clause        [[  2-phrase        (     3.0 choices)
edge  2-phrase        [[  3-word          (     1.5 choices)
edge  2-phrase        [[  4-word          (     1.6 choices)
edge  3-word          <   4-word          (213292.0 choices)
edge  1-clause        [[  5-phrase        (     3.3 choi

As you see, we have a hefty search space here.
Let us play with the `count()` function.

In [18]:
S.count(progress=10, limit=100)

  0.00s Counting results per 10 up to 100 ...
   |     0.08s 10
   |     0.08s 20
   |     0.08s 30
   |     0.10s 40
   |     0.10s 50
   |     0.10s 60
   |     0.11s 70
   |     0.11s 80
   |     0.11s 90
   |     0.11s 100
  0.11s Done: 100 results


We can be bolder than this!

In [19]:
S.count(progress=100, limit=1000)

  0.00s Counting results per 100 up to 1000 ...
   |     0.09s 100
   |     0.12s 200
   |     0.13s 300
   |     0.24s 400
   |     0.27s 500
   |     0.28s 600
   |     0.32s 700
   |     0.43s 800
   |     0.43s 900
   |     0.58s 1000
  0.58s Done: 1000 results


OK, not too bad, but note that it takes a big fraction of a second to get just 100 results.

Now let us go for all of them by the thousand.

In [20]:
S.count(progress=1000, limit=-1)

  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.53s 1000
   |     0.93s 2000
   |     1.38s 3000
   |     1.78s 4000
   |     2.14s 5000
   |     2.86s 6000
   |     4.30s 7000
  5.31s Done: 7618 results


See? This is substantial work.

In [21]:
A.table(S.fetch(limit=10))

n | p | verse | clause | phrase | word | word | phrase | word
--- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:25">Genesis 2:25</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25" sec="Genesis 2:25">Genesis 2:25</a><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hl hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span>וְלֹ֖א יִתְבֹּשָֽׁשׁוּ׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hl hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span></span>|<span class="hb"><span  class="hl hlup" >הָֽ</span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:25">Genesis 2:25</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25" sec="Genesis 2:25">Genesis 2:25</a><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hl hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span>וְלֹ֖א יִתְבֹּשָֽׁשׁוּ׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hl hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hlup" >הָֽ</span><span  class="hl hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span></span>|<span class="hb"><span  class="hl hlup" >אָדָ֖ם </span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:25">Genesis 2:25</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25" sec="Genesis 2:25">Genesis 2:25</a><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hl hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span>וְלֹ֖א יִתְבֹּשָֽׁשׁוּ׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hl hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hl hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span></span>|<span class="hb"><span  class="hl hlup" >וְ</span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:25">Genesis 2:25</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25" sec="Genesis 2:25">Genesis 2:25</a><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hl hlup" >אִשְׁתֹּ֑ו </span>וְלֹ֖א יִתְבֹּשָֽׁשׁוּ׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hl hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hl hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span></span>|<span class="hb"><span  class="hl hlup" >אִשְׁתֹּ֑ו </span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 4:4">Genesis 4:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4" sec="Genesis 4:4">Genesis 4:4</a><span  class="hlup" >וְ</span><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hl hlup" >הֵבִ֥יא </span><span  class="hl hlup" >גַם־</span><span  class="hlup" >ה֛וּא </span><span  class="hlup" >מִ</span><span  class="hlup" >בְּכֹרֹ֥ות </span><span  class="hlup" >צֹאנֹ֖ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵֽ</span><span  class="hlup" >חֶלְבֵהֶ֑ן </span>וַיִּ֣שַׁע יְהוָ֔ה אֶל־הֶ֖בֶל וְאֶל־מִנְחָתֹֽו׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hl hlup" >הֵבִ֥יא </span><span  class="hl hlup" >גַם־</span><span  class="hlup" >ה֛וּא </span><span  class="hlup" >מִ</span><span  class="hlup" >בְּכֹרֹ֥ות </span><span  class="hlup" >צֹאנֹ֖ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵֽ</span><span  class="hlup" >חֶלְבֵהֶ֑ן </span></span>|<span class="hb"><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hl hlup" >גַם־</span><span  class="hlup" >ה֛וּא </span></span>|<span class="hb"><span  class="hl hlup" >הֶ֨בֶל </span></span>|<span class="hb"><span  class="hl hlup" >גַם־</span></span>|<span class="hb"><span  class="hl hlup" >הֵבִ֥יא </span></span>|<span class="hb"><span  class="hl hlup" >הֵבִ֥יא </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 4:4">Genesis 4:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4" sec="Genesis 4:4">Genesis 4:4</a><span  class="hlup" >וְ</span><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hl hlup" >הֵבִ֥יא </span><span  class="hlup" >גַם־</span><span  class="hl hlup" >ה֛וּא </span><span  class="hlup" >מִ</span><span  class="hlup" >בְּכֹרֹ֥ות </span><span  class="hlup" >צֹאנֹ֖ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵֽ</span><span  class="hlup" >חֶלְבֵהֶ֑ן </span>וַיִּ֣שַׁע יְהוָ֔ה אֶל־הֶ֖בֶל וְאֶל־מִנְחָתֹֽו׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hl hlup" >הֵבִ֥יא </span><span  class="hlup" >גַם־</span><span  class="hl hlup" >ה֛וּא </span><span  class="hlup" >מִ</span><span  class="hlup" >בְּכֹרֹ֥ות </span><span  class="hlup" >צֹאנֹ֖ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵֽ</span><span  class="hlup" >חֶלְבֵהֶ֑ן </span></span>|<span class="hb"><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hlup" >גַם־</span><span  class="hl hlup" >ה֛וּא </span></span>|<span class="hb"><span  class="hl hlup" >הֶ֨בֶל </span></span>|<span class="hb"><span  class="hl hlup" >ה֛וּא </span></span>|<span class="hb"><span  class="hl hlup" >הֵבִ֥יא </span></span>|<span class="hb"><span  class="hl hlup" >הֵבִ֥יא </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:21">Genesis 10:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21" sec="Genesis 10:21">Genesis 10:21</a>וּלְשֵׁ֥ם יֻלַּ֖ד <span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hl hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hl hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span></span>|<span class="hb"><span  class="hl hlup" >אֲחִ֖י </span></span>|<span class="hb"><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hl hlup" >עֵ֔בֶר </span></span>|<span class="hb"><span  class="hl hlup" >עֵ֔בֶר </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:21">Genesis 10:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21" sec="Genesis 10:21">Genesis 10:21</a>וּלְשֵׁ֥ם יֻלַּ֖ד <span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span></span>|<span class="hb"><span  class="hl hlup" >אֲחִ֖י </span></span>|<span class="hb"><span  class="hl hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span></span>|<span class="hb"><span  class="hl hlup" >אֲבִי֙ </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:21">Genesis 10:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21" sec="Genesis 10:21">Genesis 10:21</a>וּלְשֵׁ֥ם יֻלַּ֖ד <span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hl hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hl hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span></span>|<span class="hb"><span  class="hl hlup" >אֲחִ֖י </span></span>|<span class="hb"><span  class="hlup" >אֲבִי֙ </span><span  class="hl hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span></span>|<span class="hb"><span  class="hl hlup" >כָּל־</span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:21">Genesis 10:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21" sec="Genesis 10:21">Genesis 10:21</a>וּלְשֵׁ֥ם יֻלַּ֖ד <span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hl hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hl hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hl hlup" >גַּם־</span></span>|<span class="hb"><span  class="hl hlup" >אֲחִ֖י </span></span>|<span class="hb"><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hl hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span></span>|<span class="hb"><span  class="hl hlup" >בְּנֵי־</span></span>

## Hand-coding

As a check, here is some code that looks for basically the same phenomenon:
a phrase within the gap of another phrase.
It does not use search, and it gets a bit more focused results, in half the time compared
to the search with the template.

> **Hint**
If you are comfortable with programming, and what you look for is fairly generic,
you may be better off without search, provided you can translate your insight in the
data into an effective procedure within Text-Fabric.
But wait till we are completely done with this example!

In [22]:
indent(reset=True)
info('Getting gapped phrases')
results = []
for v in F.otype.s('verse'):
    for c in L.d(v, otype='clause'):
        ps = L.d(c, otype='phrase')
        first = {}
        last = {}
        slots = {}
        # make index of phrase boundaries
        for p in ps:
            words = L.d(p, otype='word')
            first[p] = words[0]
            last[p] = words[-1]
            slots[p] = set(words)
        for p1 in ps:
            for p2 in ps:
                if p2 < p1: continue
                if len(slots[p1] & slots[p2]) != 0: continue
                if first[p1] < first[p2] and last[p2] < last[p1]:
                    results.append((v, c, p1, p2, first[p1], first[p2], last[p2], last[p1]))
info('{} results'.format(len(results)))

  0.00s Getting gapped phrases
  1.84s 368 results


## Pretty printing

We can use the pretty printing of `A.table()` and `A.show()` here as well, even though we have
not used search!

Not that you can show the node numbers. In this case it helps to see where the gaps are.

In [23]:
A.table(results, withNodes=True, end=10)
A.show(results, start=1, end=1)

n | p | verse | clause | phrase | phrase | word | word | word | word
--- | --- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:25">Genesis 2:25</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25" sec="Genesis 2:25">Genesis 2:25</a><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hl hlup" >אִשְׁתֹּ֑ו </span>וְלֹ֖א יִתְבֹּשָֽׁשׁוּ׃ </span> *1414245* |<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽהְי֤וּ </span><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hl hlup" >עֲרוּמִּ֔ים </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hl hlup" >אִשְׁתֹּ֑ו </span></span> *427767* |<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >וְ</span><span  class="hl hlup" >אִשְׁתֹּ֑ו </span></span> *652147* |<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span> *652148* |<span class="hb"><span  class="hl hlup" >שְׁנֵיהֶם֙ </span></span> *1159* |<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span> *1160* |<span class="hb"><span  class="hl hlup" >עֲרוּמִּ֔ים </span></span> *1160* |<span class="hb"><span  class="hl hlup" >אִשְׁתֹּ֑ו </span></span> *1164* 
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 4:4">Genesis 4:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4" sec="Genesis 4:4">Genesis 4:4</a><span  class="hlup" >וְ</span><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hl hlup" >הֵבִ֥יא </span><span  class="hlup" >גַם־</span><span  class="hl hlup" >ה֛וּא </span><span  class="hlup" >מִ</span><span  class="hlup" >בְּכֹרֹ֥ות </span><span  class="hlup" >צֹאנֹ֖ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵֽ</span><span  class="hlup" >חֶלְבֵהֶ֑ן </span>וַיִּ֣שַׁע יְהוָ֔ה אֶל־הֶ֖בֶל וְאֶל־מִנְחָתֹֽו׃ </span> *1414273* |<span class="hb"><span  class="hlup" >וְ</span><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hl hlup" >הֵבִ֥יא </span><span  class="hlup" >גַם־</span><span  class="hl hlup" >ה֛וּא </span><span  class="hlup" >מִ</span><span  class="hlup" >בְּכֹרֹ֥ות </span><span  class="hlup" >צֹאנֹ֖ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵֽ</span><span  class="hlup" >חֶלְבֵהֶ֑ן </span></span> *427889* |<span class="hb"><span  class="hl hlup" >הֶ֨בֶל </span><span  class="hlup" >גַם־</span><span  class="hl hlup" >ה֛וּא </span></span> *652504* |<span class="hb"><span  class="hl hlup" >הֵבִ֥יא </span></span> *652505* |<span class="hb"><span  class="hl hlup" >הֶ֨בֶל </span></span> *1720* |<span class="hb"><span  class="hl hlup" >הֵבִ֥יא </span></span> *1721* |<span class="hb"><span  class="hl hlup" >הֵבִ֥יא </span></span> *1721* |<span class="hb"><span  class="hl hlup" >ה֛וּא </span></span> *1723* 
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:21">Genesis 10:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21" sec="Genesis 10:21">Genesis 10:21</a>וּלְשֵׁ֥ם יֻלַּ֖ד <span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hl hlup" >עֵ֔בֶר </span><span  class="hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >גָּדֹֽול׃ </span></span> *1414445* |<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hl hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hl hlup" >עֵ֔בֶר </span><span  class="hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >גָּדֹֽול׃ </span></span> *428386* |<span class="hb"><span  class="hl hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >גָּדֹֽול׃ </span></span> *654102* |<span class="hb"><span  class="hl hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hl hlup" >עֵ֔בֶר </span></span> *654103* |<span class="hb"><span  class="hl hlup" >גַּם־</span></span> *4819* |<span class="hb"><span  class="hl hlup" >אֲבִי֙ </span></span> *4821* |<span class="hb"><span  class="hl hlup" >עֵ֔בֶר </span></span> *4824* |<span class="hb"><span  class="hl hlup" >גָּדֹֽול׃ </span></span> *4828* 
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=12&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 12:17">Genesis 12:17</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=12&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 12:17" sec="Genesis 12:17"><span  class="hlup" >Genesis 12:17</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְנַגַּ֨ע </span><span  class="hlup" >יְהוָ֧ה׀ </span><span  class="hl hlup" >אֶת־</span><span  class="hlup" >פַּרְעֹ֛ה </span><span  class="hl hlup" >נְגָעִ֥ים </span><span  class="hl hlup" >גְּדֹלִ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hl hlup" >בֵּיתֹ֑ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >דְּבַ֥ר </span><span  class="hlup" >שָׂרַ֖י </span><span  class="hlup" >אֵ֥שֶׁת </span><span  class="hlup" >אַבְרָֽם׃ </span></span> *1414505* |<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְנַגַּ֨ע </span><span  class="hlup" >יְהוָ֧ה׀ </span><span  class="hl hlup" >אֶת־</span><span  class="hlup" >פַּרְעֹ֛ה </span><span  class="hl hlup" >נְגָעִ֥ים </span><span  class="hl hlup" >גְּדֹלִ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hl hlup" >בֵּיתֹ֑ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >דְּבַ֥ר </span><span  class="hlup" >שָׂרַ֖י </span><span  class="hlup" >אֵ֥שֶׁת </span><span  class="hlup" >אַבְרָֽם׃ </span></span> *428569* |<span class="hb"><span  class="hl hlup" >אֶת־</span><span  class="hlup" >פַּרְעֹ֛ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hl hlup" >בֵּיתֹ֑ו </span></span> *654678* |<span class="hb"><span  class="hl hlup" >נְגָעִ֥ים </span><span  class="hl hlup" >גְּדֹלִ֖ים </span></span> *654679* |<span class="hb"><span  class="hl hlup" >אֶת־</span></span> *5803* |<span class="hb"><span  class="hl hlup" >נְגָעִ֥ים </span></span> *5805* |<span class="hb"><span  class="hl hlup" >גְּדֹלִ֖ים </span></span> *5806* |<span class="hb"><span  class="hl hlup" >בֵּיתֹ֑ו </span></span> *5809* 
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 13:1">Genesis 13:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 13:1" sec="Genesis 13:1">Genesis 13:1</a><span  class="hlup" >וַ</span><span  class="hlup" >יַּעַל֩ </span><span  class="hl hlup" >אַבְרָ֨ם </span><span  class="hl hlup" >מִ</span><span  class="hl hlup" >מִּצְרַ֜יִם </span><span  class="hlup" >ה֠וּא </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֧ו </span><span  class="hlup" >וְ</span><span  class="hl hlup" >כָל־</span>אֲשֶׁר־לֹ֛ו וְלֹ֥וט עִמֹּ֖ו <span  class="hlup" >הַ</span><span  class="hlup" >נֶּֽגְבָּה׃ </span></span> *1414509* |<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעַל֩ </span><span  class="hl hlup" >אַבְרָ֨ם </span><span  class="hl hlup" >מִ</span><span  class="hl hlup" >מִּצְרַ֜יִם </span><span  class="hlup" >ה֠וּא </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֧ו </span><span  class="hlup" >וְ</span><span  class="hl hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נֶּֽגְבָּה׃ </span></span> *428585* |<span class="hb"><span  class="hl hlup" >אַבְרָ֨ם </span><span  class="hlup" >ה֠וּא </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֧ו </span><span  class="hlup" >וְ</span><span  class="hl hlup" >כָל־</span></span> *654725* |<span class="hb"><span  class="hl hlup" >מִ</span><span  class="hl hlup" >מִּצְרַ֜יִם </span></span> *654726* |<span class="hb"><span  class="hl hlup" >אַבְרָ֨ם </span></span> *5868* |<span class="hb"><span  class="hl hlup" >מִ</span></span> *5869* |<span class="hb"><span  class="hl hlup" >מִּצְרַ֜יִם </span></span> *5870* |<span class="hb"><span  class="hl hlup" >כָל־</span></span> *5875* 
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 14:16">Genesis 14:16</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:16" sec="Genesis 14:16">Genesis 14:16</a>וַיָּ֕שֶׁב אֵ֖ת כָּל־הָרְכֻ֑שׁ <span  class="hlup" >וְ</span><span  class="hl hlup" >גַם֩ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >לֹ֨וט </span><span  class="hlup" >אָחִ֤יו </span><span  class="hlup" >וּ</span><span  class="hlup" >רְכֻשֹׁו֙ </span><span  class="hl hlup" >הֵשִׁ֔יב </span><span  class="hlup" >וְ</span><span  class="hlup" >גַ֥ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ</span><span  class="hl hlup" >עָֽם׃ </span></span> *1414542* |<span class="hb"><span  class="hlup" >וְ</span><span  class="hl hlup" >גַם֩ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >לֹ֨וט </span><span  class="hlup" >אָחִ֤יו </span><span  class="hlup" >וּ</span><span  class="hlup" >רְכֻשֹׁו֙ </span><span  class="hl hlup" >הֵשִׁ֔יב </span><span  class="hlup" >וְ</span><span  class="hlup" >גַ֥ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ</span><span  class="hl hlup" >עָֽם׃ </span></span> *428692* |<span class="hb"><span  class="hl hlup" >גַם֩ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >לֹ֨וט </span><span  class="hlup" >אָחִ֤יו </span><span  class="hlup" >וּ</span><span  class="hlup" >רְכֻשֹׁו֙ </span><span  class="hlup" >וְ</span><span  class="hlup" >גַ֥ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ</span><span  class="hl hlup" >עָֽם׃ </span></span> *655061* |<span class="hb"><span  class="hl hlup" >הֵשִׁ֔יב </span></span> *655062* |<span class="hb"><span  class="hl hlup" >גַם֩ </span></span> *6515* |<span class="hb"><span  class="hl hlup" >הֵשִׁ֔יב </span></span> *6521* |<span class="hb"><span  class="hl hlup" >הֵשִׁ֔יב </span></span> *6521* |<span class="hb"><span  class="hl hlup" >עָֽם׃ </span></span> *6530* 
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 17:7">Genesis 17:7</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:7" sec="Genesis 17:7">Genesis 17:7</a>וַהֲקִמֹתִ֨י אֶת־בְּרִיתִ֜י בֵּינִ֣י וּבֵינֶ֗ךָ וּבֵ֨ין זַרְעֲךָ֧ אַחֲרֶ֛יךָ לְדֹרֹתָ֖ם לִבְרִ֣ית עֹולָ֑ם <span  class="hlup" >לִ</span><span  class="hlup" >הְיֹ֤ות </span><span  class="hl hlup" >לְךָ֙ </span><span  class="hl hlup" >לֵֽ</span><span  class="hl hlup" >אלֹהִ֔ים </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >זַרְעֲךָ֖ </span><span  class="hl hlup" >אַחֲרֶֽיךָ׃ </span></span> *1414594* |<span class="hb"><span  class="hlup" >לִ</span><span  class="hlup" >הְיֹ֤ות </span><span  class="hl hlup" >לְךָ֙ </span><span  class="hl hlup" >לֵֽ</span><span  class="hl hlup" >אלֹהִ֔ים </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >זַרְעֲךָ֖ </span><span  class="hl hlup" >אַחֲרֶֽיךָ׃ </span></span> *428886* |<span class="hb"><span  class="hl hlup" >לְךָ֙ </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >זַרְעֲךָ֖ </span><span  class="hl hlup" >אַחֲרֶֽיךָ׃ </span></span> *655642* |<span class="hb"><span  class="hl hlup" >לֵֽ</span><span  class="hl hlup" >אלֹהִ֔ים </span></span> *655643* |<span class="hb"><span  class="hl hlup" >לְךָ֙ </span></span> *7431* |<span class="hb"><span  class="hl hlup" >לֵֽ</span></span> *7432* |<span class="hb"><span  class="hl hlup" >אלֹהִ֔ים </span></span> *7433* |<span class="hb"><span  class="hl hlup" >אַחֲרֶֽיךָ׃ </span></span> *7437* 
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 19:4">Genesis 19:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4" sec="Genesis 19:4">Genesis 19:4</a>טֶרֶם֮ יִשְׁכָּבוּ֒ <span  class="hlup" >וְ</span><span  class="hl hlup" >אַנְשֵׁ֨י </span><span  class="hlup" >הָ</span><span  class="hlup" >עִ֜יר </span><span  class="hlup" >אַנְשֵׁ֤י </span><span  class="hlup" >סְדֹם֙ </span><span  class="hl hlup" >נָסַ֣בּוּ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הַ</span><span  class="hlup" >בַּ֔יִת </span><span  class="hlup" >מִ</span><span  class="hlup" >נַּ֖עַר </span><span  class="hlup" >וְ</span><span  class="hlup" >עַד־</span><span  class="hlup" >זָקֵ֑ן </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֖ם </span><span  class="hlup" >מִ</span><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *1414651* |<span class="hb"><span  class="hlup" >וְ</span><span  class="hl hlup" >אַנְשֵׁ֨י </span><span  class="hlup" >הָ</span><span  class="hlup" >עִ֜יר </span><span  class="hlup" >אַנְשֵׁ֤י </span><span  class="hlup" >סְדֹם֙ </span><span  class="hl hlup" >נָסַ֣בּוּ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הַ</span><span  class="hlup" >בַּ֔יִת </span><span  class="hlup" >מִ</span><span  class="hlup" >נַּ֖עַר </span><span  class="hlup" >וְ</span><span  class="hlup" >עַד־</span><span  class="hlup" >זָקֵ֑ן </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֖ם </span><span  class="hlup" >מִ</span><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *429128* |<span class="hb"><span  class="hl hlup" >אַנְשֵׁ֨י </span><span  class="hlup" >הָ</span><span  class="hlup" >עִ֜יר </span><span  class="hlup" >אַנְשֵׁ֤י </span><span  class="hlup" >סְדֹם֙ </span><span  class="hlup" >מִ</span><span  class="hlup" >נַּ֖עַר </span><span  class="hlup" >וְ</span><span  class="hlup" >עַד־</span><span  class="hlup" >זָקֵ֑ן </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֖ם </span><span  class="hlup" >מִ</span><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *656353* |<span class="hb"><span  class="hl hlup" >נָסַ֣בּוּ </span></span> *656354* |<span class="hb"><span  class="hl hlup" >אַנְשֵׁ֨י </span></span> *8502* |<span class="hb"><span  class="hl hlup" >נָסַ֣בּוּ </span></span> *8507* |<span class="hb"><span  class="hl hlup" >נָסַ֣בּוּ </span></span> *8507* |<span class="hb"><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *8520* 
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 19:4">Genesis 19:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4" sec="Genesis 19:4">Genesis 19:4</a>טֶרֶם֮ יִשְׁכָּבוּ֒ <span  class="hlup" >וְ</span><span  class="hl hlup" >אַנְשֵׁ֨י </span><span  class="hlup" >הָ</span><span  class="hlup" >עִ֜יר </span><span  class="hlup" >אַנְשֵׁ֤י </span><span  class="hlup" >סְדֹם֙ </span><span  class="hlup" >נָסַ֣בּוּ </span><span  class="hl hlup" >עַל־</span><span  class="hlup" >הַ</span><span  class="hl hlup" >בַּ֔יִת </span><span  class="hlup" >מִ</span><span  class="hlup" >נַּ֖עַר </span><span  class="hlup" >וְ</span><span  class="hlup" >עַד־</span><span  class="hlup" >זָקֵ֑ן </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֖ם </span><span  class="hlup" >מִ</span><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *1414651* |<span class="hb"><span  class="hlup" >וְ</span><span  class="hl hlup" >אַנְשֵׁ֨י </span><span  class="hlup" >הָ</span><span  class="hlup" >עִ֜יר </span><span  class="hlup" >אַנְשֵׁ֤י </span><span  class="hlup" >סְדֹם֙ </span><span  class="hlup" >נָסַ֣בּוּ </span><span  class="hl hlup" >עַל־</span><span  class="hlup" >הַ</span><span  class="hl hlup" >בַּ֔יִת </span><span  class="hlup" >מִ</span><span  class="hlup" >נַּ֖עַר </span><span  class="hlup" >וְ</span><span  class="hlup" >עַד־</span><span  class="hlup" >זָקֵ֑ן </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֖ם </span><span  class="hlup" >מִ</span><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *429128* |<span class="hb"><span  class="hl hlup" >אַנְשֵׁ֨י </span><span  class="hlup" >הָ</span><span  class="hlup" >עִ֜יר </span><span  class="hlup" >אַנְשֵׁ֤י </span><span  class="hlup" >סְדֹם֙ </span><span  class="hlup" >מִ</span><span  class="hlup" >נַּ֖עַר </span><span  class="hlup" >וְ</span><span  class="hlup" >עַד־</span><span  class="hlup" >זָקֵ֑ן </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֖ם </span><span  class="hlup" >מִ</span><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *656353* |<span class="hb"><span  class="hl hlup" >עַל־</span><span  class="hlup" >הַ</span><span  class="hl hlup" >בַּ֔יִת </span></span> *656355* |<span class="hb"><span  class="hl hlup" >אַנְשֵׁ֨י </span></span> *8502* |<span class="hb"><span  class="hl hlup" >עַל־</span></span> *8508* |<span class="hb"><span  class="hl hlup" >בַּ֔יִת </span></span> *8510* |<span class="hb"><span  class="hl hlup" >קָּצֶֽה׃ </span></span> *8520* 
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 22:3">Genesis 22:3</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:3" sec="Genesis 22:3">Genesis 22:3</a>וַיַּשְׁכֵּ֨ם אַבְרָהָ֜ם בַּבֹּ֗קֶר וַֽיַּחֲבֹשׁ֙ אֶת־חֲמֹרֹ֔ו <span  class="hlup" >וַ</span><span  class="hlup" >יִּקַּ֞ח </span><span  class="hl hlup" >אֶת־</span><span  class="hlup" >שְׁנֵ֤י </span><span  class="hlup" >נְעָרָיו֙ </span><span  class="hl hlup" >אִתֹּ֔ו </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֖ת </span><span  class="hlup" >יִצְחָ֣ק </span><span  class="hl hlup" >בְּנֹ֑ו </span>וַיְבַקַּע֙ עֲצֵ֣י עֹלָ֔ה וַיָּ֣קָם וַיֵּ֔לֶךְ אֶל־הַמָּקֹ֖ום אֲשֶׁר־אָֽמַר־לֹ֥ו הָאֱלֹהִֽים׃ </span> *1414740* |<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּקַּ֞ח </span><span  class="hl hlup" >אֶת־</span><span  class="hlup" >שְׁנֵ֤י </span><span  class="hlup" >נְעָרָיו֙ </span><span  class="hl hlup" >אִתֹּ֔ו </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֖ת </span><span  class="hlup" >יִצְחָ֣ק </span><span  class="hl hlup" >בְּנֹ֑ו </span></span> *429497* |<span class="hb"><span  class="hl hlup" >אֶת־</span><span  class="hlup" >שְׁנֵ֤י </span><span  class="hlup" >נְעָרָיו֙ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֖ת </span><span  class="hlup" >יִצְחָ֣ק </span><span  class="hl hlup" >בְּנֹ֑ו </span></span> *657505* |<span class="hb"><span  class="hl hlup" >אִתֹּ֔ו </span></span> *657506* |<span class="hb"><span  class="hl hlup" >אֶת־</span></span> *10284* |<span class="hb"><span  class="hl hlup" >אִתֹּ֔ו </span></span> *10287* |<span class="hb"><span  class="hl hlup" >אִתֹּ֔ו </span></span> *10287* |<span class="hb"><span  class="hl hlup" >בְּנֹ֑ו </span></span> *10291* 



**verse** *1*



**NB**
Gaps are a tricky phenomenon. In [gaps](searchGaps.ipynb) we will deal with them cruelly.

## An other gap example

Here is an example by Yanniek van der Schans (2018-09-21).

In [24]:
query = '''
c:clause
 PreGap:phrase_atom
 LastPhrase:phrase_atom
 :=

Gap:clause_atom
 :: word

PreGap < Gap
Gap < LastPhrase
c || Gap
'''

In [25]:
S.study(query)

   |     0.00s Feature overview: 111 for nodes; 8 for edges; 2 configs; 7 computed
  0.00s Checking search template ...
  0.00s Setting up search space for 5 objects ...
  0.19s Constraining search space with 8 relations ...
  0.27s Setting up retrieval plan ...
  0.28s Ready to deliver results from 633541 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


In [26]:
S.showPlan(details=True)

Search with 5 objects and 8 relations
Results are instantiations of the following objects:
node  0-clause                            ( 88101   choices)
node  1-phrase_atom                       (267519   choices)
node  2-phrase_atom                       (267519   choices)
node  3-clause_atom                       (  5201   choices)
node  4-word                              (  5201   choices)
Instantiations are computed along the following relations:
node                      3-clause_atom   (  5201   choices)
edge  3-clause_atom   ::  4-word          (     1.0 choices)
edge  4-word          ]]  3-clause_atom   (     1.0 choices)
edge  3-clause_atom   ||  0-clause        ( 79290.9 choices)
edge  0-clause        :=  2-phrase_atom   (     1.0 choices)
edge  2-phrase_atom   ]]  0-clause        (     1.0 choices)
edge  2-phrase_atom   >   3-clause_atom   (  2600.5 choices)
edge  0-clause        [[  1-phrase_atom   (     2.9 choices)
edge  1-phrase_atom   <   3-clause_atom   (  2600.5 choic

In [27]:
S.count(progress=1, limit=1)

  0.00s Counting results per 1 up to 1 ...
   |     4.15s 1
  4.15s Done: 1 results


# Next

You have seen cases where the implementation is to blame.

Now I want to point to gaps in your understanding:
[gaps](searchGaps.ipynb)

---

[basic](search.ipynb)
[advanced](searchAdvanced.ipynb)
[sets](searchSets.ipynb)
[relations](searchRelations.ipynb)
[quantifiers](searchQuantifiers.ipynb)
rough
[gaps](searchGaps.ipynb)